In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../datasets/sep28k/SEP-28k_labels_new.csv')
print("Total number of samples: ",df.shape)
df.head(5)

Total number of samples:  (21856, 17)


,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech
0,HeStutters,0,0,31900320,31948320,0,0,0,0,0,0,0,0,3,1,0,0
1,HeStutters,0,1,31977120,32025120,0,0,0,0,0,0,0,0,3,1,0,0
2,HeStutters,0,2,34809760,34857760,0,0,0,0,0,0,0,0,3,0,0,0
3,HeStutters,0,3,35200640,35248640,0,0,1,0,0,0,0,0,2,0,0,0
4,HeStutters,0,4,35721920,35769920,0,0,0,0,0,0,0,0,3,0,0,0


In [3]:
label_cols = ['Prolongation', 'Block', 'SoundRep', 'WordRep', 'Interjection', 'NoStutteredWords']
df['label_per_type'] = df[label_cols].apply(lambda row: len(label_cols) if row.max() <= 1 else label_cols.index(row.idxmax()), axis=1)
df['label_per_type'].value_counts()


label_per_type
5    9550
4    3311
1    2201
3    2053
0    1800
2    1594
6    1347
Name: count, dtype: int64

In [4]:
# display samples with label_per_type == 4 and NoStutteredWords >= 2
df[(df['label_per_type'] == 4) & (df['NoStutteredWords'] >= 2)]

,Show,EpId,ClipId,Start,Stop,Unsure,PoorAudioQuality,Prolongation,Block,SoundRep,WordRep,DifficultToUnderstand,Interjection,NoStutteredWords,NaturalPause,Music,NoSpeech,label_per_type
43,HeStutters,1,3,28193760,28241760,0,0,0,1,0,0,0,3,2,1,0,0,4
66,HeStutters,1,26,30961440,31009440,0,0,0,1,0,0,0,2,2,0,0,0,4
68,HeStutters,1,28,31132480,31180480,0,1,1,0,0,1,1,2,2,0,0,0,4
76,HeStutters,1,36,31917440,31965440,0,0,0,1,0,0,0,3,2,0,0,0,4
83,HeStutters,1,43,32695840,32743840,0,0,0,1,0,0,0,3,2,2,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21843,WomenWhoStutter,109,27,16899200,16947200,0,0,0,0,0,1,0,3,2,1,0,0,4
21848,WomenWhoStutter,109,32,21712320,21760320,0,0,1,0,0,0,0,2,2,0,0,0,4
21851,WomenWhoStutter,109,35,25793600,25841600,0,0,0,0,0,0,0,3,3,0,0,0,4
21853,WomenWhoStutter,109,37,3569440,3617440,0,0,0,0,0,0,0,3,3,1,0,0,4


In [5]:
from statsmodels.stats import inter_rater as irr
label_df = df.iloc[:,5:-1]
label_df
for col in label_df.columns:
    temp = pd.DataFrame()
    temp['pos'] = label_df[col]
    temp['neg'] = temp['pos'].apply(lambda x: 3-x )
    # compute Fleiss' Kappa for the current category
    fleiss_kappa = irr.fleiss_kappa(temp.values, method='fleiss')
    print(f"Fleiss' Kappa for category {col}: {fleiss_kappa:.2f}")


Fleiss' Kappa for category Unsure: 0.02
Fleiss' Kappa for category PoorAudioQuality: 0.19
Fleiss' Kappa for category Prolongation: 0.24
Fleiss' Kappa for category Block: 0.12
Fleiss' Kappa for category SoundRep: 0.41
Fleiss' Kappa for category WordRep: 0.64
Fleiss' Kappa for category DifficultToUnderstand: 0.16
Fleiss' Kappa for category Interjection: 0.57
Fleiss' Kappa for category NoStutteredWords: 0.40
Fleiss' Kappa for category NaturalPause: 0.09
Fleiss' Kappa for category Music: 0.80
Fleiss' Kappa for category NoSpeech: 0.37


In [29]:
df['Unsure'].value_counts()

Unsure
0    21273
1      566
2       17
Name: count, dtype: int64